In [1]:
import numpy as np
import os
import time
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model,Sequential
from keras.utils import np_utils
from sklearn import preprocessing
from keras.layers import *
from keras.models import *
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from keras.layers import Convolution2D, GlobalAveragePooling2D,Dropout
from keras.layers.recurrent import LSTM
import glob
from keras import optimizers
#from keras.applications.resnet50 import ResNet50

Using TensorFlow backend.


In [2]:
PATH = '../biswa_n_deba/FINAL_new'
PATH1 = '../biswa_n_deba/FINAL_new'
# Define data path
data_path = PATH + '/FINAL_TRAIN_MAG'
data_path1 = PATH1 + '/Train'
data_dir_list = os.listdir(data_path)
data_dir_list1 = os.listdir(data_path1)

In [3]:
data_dir_list.sort()
# data_dir_list.pop(8)
# data_dir_list.pop(8)
# data_dir_list.pop(3)
#data_dir_list.pop(0)
data_dir_list

['Archery-mag.csv',
 'BaseballPitch-mag.csv',
 'Biking-mag.csv',
 'Bowling-mag.csv',
 'BoxingPunchingBag-mag.csv',
 'BreastStroke-mag.csv',
 'CricketBowling-mag.csv',
 'Diving-mag.csv',
 'HighJump-mag.csv',
 'HorseRiding-mag.csv',
 'IceDancing-mag.csv',
 'JavelinThrow-mag.csv',
 'PoleVault-mag.csv',
 'Rafting-mag.csv',
 'RopeClimbing-mag.csv',
 'Shotput-mag.csv',
 'SkateBoarding-mag.csv',
 'Surfing-mag.csv',
 'TennisSwing-mag.csv',
 'VolleyballSpiking-mag.csv']

In [4]:
data_dir_list1.sort()
#data_dir_list1.pop(0)
data_dir_list1

['Archery',
 'BaseballPitch',
 'Biking',
 'Bowling',
 'BoxingPunchingBag',
 'BreastStroke',
 'CricketBowling',
 'Diving',
 'HighJump',
 'HorseRiding',
 'IceDancing',
 'JavelinThrow',
 'PoleVault',
 'Rafting',
 'RopeClimbing',
 'Shotput',
 'SkateBoarding',
 'Surfing',
 'TennisSwing',
 'VolleyballSpiking']

In [5]:
__clip = '../biswa_n_deba/FINAL_new/Train/'

actionClasses = ['Archery',
 'BaseballPitch',
 'Biking',
 'Bowling',
 'BoxingPunchingBag',
 'BreastStroke',
 'CricketBowling',
 'Diving',
 'HighJump',
 'HorseRiding',
 'IceDancing',
 'JavelinThrow',
 'PoleVault',
 'Rafting',
 'RopeClimbing',
 'Shotput',
 'SkateBoarding',
 'Surfing',
 'TennisSwing',
 'VolleyballSpiking'] #35
actionClasses.sort()
len_class = []
clip1 = list()
clip = list()
for i in range(len(actionClasses)):
    clip1.append( __clip + actionClasses[i] + "/*.avi")
    clip.append(sorted(glob.glob(clip1[i])))
#print(clip[0])

for i in range(len(actionClasses)):
        len_class.append(len(clip[i] * 5))

In [6]:
len_class1 = len_class
len_class = np.array(len_class)
print(len_class)

[445 455 435 450 480 375 435 465 400 495 480 405 425 390 395 445 390 420
 500 405]


In [7]:
actionClasses

['Archery',
 'BaseballPitch',
 'Biking',
 'Bowling',
 'BoxingPunchingBag',
 'BreastStroke',
 'CricketBowling',
 'Diving',
 'HighJump',
 'HorseRiding',
 'IceDancing',
 'JavelinThrow',
 'PoleVault',
 'Rafting',
 'RopeClimbing',
 'Shotput',
 'SkateBoarding',
 'Surfing',
 'TennisSwing',
 'VolleyballSpiking']

In [8]:
i = 0
for files in data_dir_list:
    print(files)
    X = pd.read_csv(data_path + '/' + files)
    X = X.iloc[:,1:].values
    a = np.shape(X[0])
    print(a[0])
    len_class[i] = len_class[i] - a[0]
    i = i+1
i = 0

Archery-mag.csv
416
BaseballPitch-mag.csv
423
Biking-mag.csv
419
Bowling-mag.csv
420
BoxingPunchingBag-mag.csv
461
BreastStroke-mag.csv
347
CricketBowling-mag.csv
389
Diving-mag.csv
450
HighJump-mag.csv
354
HorseRiding-mag.csv
466
IceDancing-mag.csv
467
JavelinThrow-mag.csv
353
PoleVault-mag.csv
398
Rafting-mag.csv
370
RopeClimbing-mag.csv
381
Shotput-mag.csv
407
SkateBoarding-mag.csv
358
Surfing-mag.csv
407
TennisSwing-mag.csv
466
VolleyballSpiking-mag.csv
379


In [9]:
print(len_class)
z=0

[29 32 16 30 19 28 46 15 46 29 13 52 27 20 14 38 32 13 34 26]


In [10]:
img_data_list = []
for files in data_dir_list:
    X = pd.read_csv(data_path + '/' + files)
    X = X.replace([np.inf, -np.inf], np.nan)
    X = X.fillna(0)
    X = X.iloc[:,1:].values
    Y = X[:,-1*len_class[z]:]
    #print(np.shape(Y))
    X = np.concatenate((X,Y),axis=1)
    print(np.shape(X))
    z = z+1
    print(files)
    for i in range(len(X[0])):
        col = X[:,i]
        #col = (col-np.amin(col))/(np.amax(col)-np.amin(col))
        col = np.reshape(col,(240,320))
        col = cv2.resize(col,(224,224),interpolation = cv2.INTER_CUBIC)
        col = np.repeat(col[:, :, np.newaxis], 3, axis=2)
        col = np.expand_dims(col, axis=0)
        col = preprocess_input(col)
        print('Input image shape:', col.shape)
        img_data_list.append(col)
        print(i)

(76800, 445)
Archery-mag.csv
('Input image shape:', (1, 224, 224, 3))
0
('Input image shape:', (1, 224, 224, 3))
1
('Input image shape:', (1, 224, 224, 3))
2
('Input image shape:', (1, 224, 224, 3))
3
('Input image shape:', (1, 224, 224, 3))
4
('Input image shape:', (1, 224, 224, 3))
5
('Input image shape:', (1, 224, 224, 3))
6
('Input image shape:', (1, 224, 224, 3))
7
('Input image shape:', (1, 224, 224, 3))
8
('Input image shape:', (1, 224, 224, 3))
9
('Input image shape:', (1, 224, 224, 3))
10
('Input image shape:', (1, 224, 224, 3))
11
('Input image shape:', (1, 224, 224, 3))
12
('Input image shape:', (1, 224, 224, 3))
13
('Input image shape:', (1, 224, 224, 3))
14
('Input image shape:', (1, 224, 224, 3))
15
('Input image shape:', (1, 224, 224, 3))
16
('Input image shape:', (1, 224, 224, 3))
17
('Input image shape:', (1, 224, 224, 3))
18
('Input image shape:', (1, 224, 224, 3))
19
('Input image shape:', (1, 224, 224, 3))
20
('Input image shape:', (1, 224, 224, 3))
21
('Input image

In [11]:
img_data = np.array(img_data_list)
print (img_data.shape)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)
img_data=img_data[0]
print (img_data.shape) 

(8690, 1, 224, 224, 3)
(1, 8690, 224, 224, 3)
(8690, 224, 224, 3)


In [12]:
for i in range(len(len_class1)-1):
    len_class1[i+1] = len_class1[i+1] + len_class1[i]
print(len_class1)

[445, 900, 1335, 1785, 2265, 2640, 3075, 3540, 3940, 4435, 4915, 5320, 5745, 6135, 6530, 6975, 7365, 7785, 8285, 8690]


In [13]:
num_classes = len(actionClasses)
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples),dtype='int64')
labels[0:len_class1[0]] = 0
print(len_class1[0])
i = 1
for i in range (1,num_classes):
    labels[len_class1[i-1] : len_class1[i]] = i
    print(len_class1[i-1],len_class1[i],i)
print(labels)

445
(445, 900, 1)
(900, 1335, 2)
(1335, 1785, 3)
(1785, 2265, 4)
(2265, 2640, 5)
(2640, 3075, 6)
(3075, 3540, 7)
(3540, 3940, 8)
(3940, 4435, 9)
(4435, 4915, 10)
(4915, 5320, 11)
(5320, 5745, 12)
(5745, 6135, 13)
(6135, 6530, 14)
(6530, 6975, 15)
(6975, 7365, 16)
(7365, 7785, 17)
(7785, 8285, 18)
(8285, 8690, 19)
[ 0  0  0 ... 19 19 19]


In [14]:
names = ['Archery',
 'BaseballPitch',
 'Biking',
 'Bowling',
 'BoxingPunchingBag',
 'BreastStroke',
 'CricketBowling',
 'Diving',
 'HighJump',
 'HorseRiding',
 'IceDancing',
 'JavelinThrow',
 'PoleVault',
 'Rafting',
 'RopeClimbing',
 'Shotput',
 'SkateBoarding',
 'Surfing',
 'TennisSwing',
 'VolleyballSpiking']

# convert class labels to on-hot encoding
Y = np_utils.to_categorical(labels, num_classes)

In [15]:
print(np.shape(Y))
print(Y)

(8690, 20)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [16]:
x_train = img_data

In [17]:
print(np.shape(x_train))

(8690, 224, 224, 3)


In [18]:
y_train = Y
print(np.shape(y_train))

(8690, 20)


In [19]:
PATH = '../biswa_n_deba/FINAL_new'
PATH1 = '../biswa_n_deba/FINAL_new'
# Define data path
data_path = PATH + '/FINAL_TEST_MAG'
data_path1 = PATH1 + '/Test'
data_dir_list = os.listdir(data_path)
data_dir_list1 = os.listdir(data_path1)

In [20]:
data_dir_list.sort()
#data_dir_list.pop(0)
data_dir_list
# data_dir_list.pop(8)
# data_dir_list.pop(3)
# data_dir_list.pop(7)
# data_dir_list

['Archery-mag.csv',
 'BaseballPitch-mag.csv',
 'Biking-mag.csv',
 'Bowling-mag.csv',
 'BoxingPunchingBag-mag.csv',
 'BreastStroke-mag.csv',
 'CricketBowling-mag.csv',
 'Diving-mag.csv',
 'HighJump-mag.csv',
 'HorseRiding-mag.csv',
 'IceDancing-mag.csv',
 'JavelinThrow-mag.csv',
 'PoleVault-mag.csv',
 'Rafting-mag.csv',
 'RopeClimbing-mag.csv',
 'Shotput-mag.csv',
 'SkateBoarding-mag.csv',
 'Surfing-mag.csv',
 'TennisSwing-mag.csv',
 'VolleyballSpiking-mag.csv']

In [21]:
__clip = '../biswa_n_deba/FINAL_new/Test/'

actionClasses = ['Archery',
 'BaseballPitch',
 'Biking',
 'Bowling',
 'BoxingPunchingBag',
 'BreastStroke',
 'CricketBowling',
 'Diving',
 'HighJump',
 'HorseRiding',
 'IceDancing',
 'JavelinThrow',
 'PoleVault',
 'Rafting',
 'RopeClimbing',
 'Shotput',
 'SkateBoarding',
 'Surfing',
 'TennisSwing',
 'VolleyballSpiking'] #35
actionClasses.sort()
len_class = []
clip1 = list()
clip = list()
for i in range(len(actionClasses)):
    clip1.append( __clip + actionClasses[i] + "/*.avi")
    clip.append(sorted(glob.glob(clip1[i])))
#print(clip[0])

for i in range(len(actionClasses)):
        len_class.append(len(clip[i] * 5))
        
len_class1 = len_class
len_class = np.array(len_class)
print(len_class)

[280 295 235 310 335 130 260 285 215 325 290 180 295 165 200 255 200 210
 330 175]


In [22]:
i = 0
for files in data_dir_list:
    print(files)
    X = pd.read_csv(data_path + '/' + files)
    X = X.iloc[:,1:].values
    a = np.shape(X[0])
    len_class[i] = len_class[i] - a[0]
    i = i+1
i = 0

print(len_class)

Archery-mag.csv
BaseballPitch-mag.csv
Biking-mag.csv
Bowling-mag.csv
BoxingPunchingBag-mag.csv
BreastStroke-mag.csv
CricketBowling-mag.csv
Diving-mag.csv
HighJump-mag.csv
HorseRiding-mag.csv
IceDancing-mag.csv
JavelinThrow-mag.csv
PoleVault-mag.csv
Rafting-mag.csv
RopeClimbing-mag.csv
Shotput-mag.csv
SkateBoarding-mag.csv
Surfing-mag.csv
TennisSwing-mag.csv
VolleyballSpiking-mag.csv
[10 21  5 25 17 10 28 12 19 17 13 25 16 15 12 26 18 10 21  9]


In [23]:
z=0
img_data_list = []
for files in data_dir_list:
    X = pd.read_csv(data_path + '/' + files)
    X = X.replace([np.inf, -np.inf], np.nan)
    X = X.fillna(0)
    X = X.iloc[:,1:].values
    Y = X[:,-1*len_class[z]:]
    #print(np.shape(Y))
    X = np.concatenate((X,Y),axis=1)
    print(np.shape(X))
    z = z+1
    print(files)
    for i in range(len(X[0])):
        col = X[:,i]
        #col = (col-np.amin(col))/(np.amax(col)-np.amin(col))
        col = np.reshape(col,(240,320))
        col = cv2.resize(col,(224,224),interpolation = cv2.INTER_CUBIC)
        col = np.repeat(col[:, :, np.newaxis], 3, axis=2)
        col = np.expand_dims(col, axis=0)
        col = preprocess_input(col)
        print('Input image shape:', col.shape)
        img_data_list.append(col)
        print(i)

(76800, 280)
Archery-mag.csv
('Input image shape:', (1, 224, 224, 3))
0
('Input image shape:', (1, 224, 224, 3))
1
('Input image shape:', (1, 224, 224, 3))
2
('Input image shape:', (1, 224, 224, 3))
3
('Input image shape:', (1, 224, 224, 3))
4
('Input image shape:', (1, 224, 224, 3))
5
('Input image shape:', (1, 224, 224, 3))
6
('Input image shape:', (1, 224, 224, 3))
7
('Input image shape:', (1, 224, 224, 3))
8
('Input image shape:', (1, 224, 224, 3))
9
('Input image shape:', (1, 224, 224, 3))
10
('Input image shape:', (1, 224, 224, 3))
11
('Input image shape:', (1, 224, 224, 3))
12
('Input image shape:', (1, 224, 224, 3))
13
('Input image shape:', (1, 224, 224, 3))
14
('Input image shape:', (1, 224, 224, 3))
15
('Input image shape:', (1, 224, 224, 3))
16
('Input image shape:', (1, 224, 224, 3))
17
('Input image shape:', (1, 224, 224, 3))
18
('Input image shape:', (1, 224, 224, 3))
19
('Input image shape:', (1, 224, 224, 3))
20
('Input image shape:', (1, 224, 224, 3))
21
('Input image

In [24]:
img_data = np.array(img_data_list)
print (img_data.shape)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)
img_data=img_data[0]
print (img_data.shape) 

(4970, 1, 224, 224, 3)
(1, 4970, 224, 224, 3)
(4970, 224, 224, 3)


In [25]:
for i in range(len(len_class1)-1):
    len_class1[i+1] = len_class1[i+1] + len_class1[i]
print(len_class1)

[280, 575, 810, 1120, 1455, 1585, 1845, 2130, 2345, 2670, 2960, 3140, 3435, 3600, 3800, 4055, 4255, 4465, 4795, 4970]


In [26]:
num_classes = len(actionClasses)
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples),dtype='int64')
labels[0:len_class1[0]] = 0
print(len_class1[0])
i = 1
for i in range (1,num_classes):
    labels[len_class1[i-1] : len_class1[i]] = i
    print(len_class1[i-1],len_class1[i],i)
print(labels)

280
(280, 575, 1)
(575, 810, 2)
(810, 1120, 3)
(1120, 1455, 4)
(1455, 1585, 5)
(1585, 1845, 6)
(1845, 2130, 7)
(2130, 2345, 8)
(2345, 2670, 9)
(2670, 2960, 10)
(2960, 3140, 11)
(3140, 3435, 12)
(3435, 3600, 13)
(3600, 3800, 14)
(3800, 4055, 15)
(4055, 4255, 16)
(4255, 4465, 17)
(4465, 4795, 18)
(4795, 4970, 19)
[ 0  0  0 ... 19 19 19]


In [27]:
names = ['Archery',
 'BaseballPitch',
 'Biking',
 'Bowling',
 'BoxingPunchingBag',
 'BreastStroke',
 'CricketBowling',
 'Diving',
 'HighJump',
 'HorseRiding',
 'IceDancing',
 'JavelinThrow',
 'PoleVault',
 'Rafting',
 'RopeClimbing',
 'Shotput',
 'SkateBoarding',
 'Surfing',
 'TennisSwing',
 'VolleyballSpiking']

# convert class labels to on-hot encoding
Y = np_utils.to_categorical(labels, num_classes)

In [28]:
x_test = img_data
y_test = Y

In [29]:
# print(x_train)
# print(y_train)
# print(x_test)
# print(y_test)
print Y

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [30]:
image_input = Input(shape=(224, 224, 3))

In [31]:
model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [32]:
last_layer = model.get_layer('fc2').output
out = Dense(num_classes, activation='softmax', name='output')(last_layer)
custom_vgg_model = Model(image_input, out)
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [33]:
custom_vgg_model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [34]:
for layer in custom_vgg_model.layers[:-1]:
    layer.trainable = False

In [35]:
custom_vgg_model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [36]:
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [37]:
t=time.time()
#	t = now()
hist = custom_vgg_model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(x_test, y_test))
print('Training time: %s' % (t - time.time()))
(loss, accuracy) = custom_vgg_model.evaluate(x_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

Train on 8690 samples, validate on 4970 samples
Epoch 1/10
8690/8690 [==============================] - 34s 4ms/step - loss: 2.4326 - acc: 0.2625 - val_loss: 2.3233 - val_acc: 0.3173
Epoch 2/10
8690/8690 [==============================] - 31s 4ms/step - loss: 1.9651 - acc: 0.3846 - val_loss: 2.2816 - val_acc: 0.3431
Epoch 3/10
8690/8690 [==============================] - 31s 4ms/step - loss: 1.7909 - acc: 0.4391 - val_loss: 2.2319 - val_acc: 0.3386
Epoch 4/10
8690/8690 [==============================] - 31s 4ms/step - loss: 1.6647 - acc: 0.4718 - val_loss: 2.1624 - val_acc: 0.3616
Epoch 5/10
8690/8690 [==============================] - 31s 4ms/step - loss: 1.5726 - acc: 0.4990 - val_loss: 2.1509 - val_acc: 0.3843
Epoch 6/10
8690/8690 [==============================] - 31s 4ms/step - loss: 1.4928 - acc: 0.5212 - val_loss: 2.5245 - val_acc: 0.3254
Epoch 7/10
8690/8690 [==============================] - 31s 4ms/step - loss: 1.4191 - acc: 0.5448 - val_loss: 2.1344 - val_acc: 0.4103
Epoch 8

In [38]:
image_input = Input(shape=(224, 224, 3))

model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [39]:
last_layer = model.get_layer('block5_pool').output
x= Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
out = Dense(num_classes, activation='softmax', name='output')(x)
custom_vgg_model2 = Model(image_input, out)
custom_vgg_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [40]:
# freeze all the layers except the dense layers
for layer in custom_vgg_model2.layers[:-3]:
    layer.trainable = False

custom_vgg_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [41]:
custom_vgg_model2.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

In [42]:
t=time.time()
#	t = now()
hist = custom_vgg_model2.fit(x_train, y_train, batch_size=32, epochs=8, verbose=1, validation_data=(x_test, y_test))
print('Training time: %s' % (t - time.time()))
(loss, accuracy) = custom_vgg_model2.evaluate(x_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

Train on 8690 samples, validate on 4970 samples
Epoch 1/8
8690/8690 [==============================] - 32s 4ms/step - loss: 2.2908 - acc: 0.3442 - val_loss: 2.0863 - val_acc: 0.3817
Epoch 2/8
8690/8690 [==============================] - 32s 4ms/step - loss: 1.2515 - acc: 0.6175 - val_loss: 2.0755 - val_acc: 0.4153
Epoch 3/8
8690/8690 [==============================] - 32s 4ms/step - loss: 0.7723 - acc: 0.7545 - val_loss: 2.0371 - val_acc: 0.4821
Epoch 4/8
8690/8690 [==============================] - 32s 4ms/step - loss: 0.4837 - acc: 0.8441 - val_loss: 2.5283 - val_acc: 0.4529
Epoch 5/8
8690/8690 [==============================] - 32s 4ms/step - loss: 0.2971 - acc: 0.9067 - val_loss: 2.4690 - val_acc: 0.4992
Epoch 6/8
8690/8690 [==============================] - 32s 4ms/step - loss: 0.1968 - acc: 0.9372 - val_loss: 2.7159 - val_acc: 0.5101
Epoch 7/8
8690/8690 [==============================] - 32s 4ms/step - loss: 0.1351 - acc: 0.9594 - val_loss: 3.1720 - val_acc: 0.4841
Epoch 8/8
8690

In [43]:
a = custom_vgg_model2.predict(x_train)

In [44]:
a[0]

array([9.95112479e-01, 5.93729492e-05, 1.56183376e-07, 6.58543664e-09,
       1.98962749e-10, 1.71780128e-14, 1.20670099e-10, 1.64511021e-05,
       2.59967581e-09, 1.63179036e-04, 3.81855649e-08, 3.84056830e-06,
       1.07965076e-04, 4.14077519e-03, 4.98952375e-07, 9.85949669e-08,
       3.95114417e-04, 1.23369293e-09, 3.44629103e-09, 2.03793027e-09],
      dtype=float32)

In [45]:
np.sum(a[2])

1.0

In [46]:
a = custom_vgg_model2.predict(x_train)
#c = model.predict(predictions)
b=np.shape(a)
#d=np.shape(c)
print(b)
print(np.shape(x_train))

(8690, 20)
(8690, 224, 224, 3)


In [47]:
a = a.reshape(1738,5,20)
print(np.shape(a))

(1738, 5, 20)


In [48]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [49]:
num_Frames = 5
num_classes = len(actionClasses)
LSTM_Model=Sequential()
LSTM_Model.add(LSTM(512, input_shape=(num_Frames,num_classes), return_sequences=False))
LSTM_Model.add(Activation('relu'))

LSTM_Model.add(Dense(256))
LSTM_Model.add(Activation('relu'))

LSTM_Model.add(Dense(num_classes))
LSTM_Model.add(Activation('softmax'))

LSTM_Model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
LSTM_Model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 512)               1091584   
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                5140      
_________________________________________________________________
activation_3 (Activation)    (None, 20)                0         
Total params: 1,228,052
Trainable params: 1,228,052
Non-trainable params: 0
_________________________________________________________________


In [50]:
print(y_train)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [51]:
# print(y_train.shape)
# y_train = np_utils.to_categorical(y_train, num_classes)
# print(y_train.shape)
# print(y_test.shape)
# y_test =np_utils.to_categorical(y_test, num_classes)
# print(y_test.shape)

In [52]:
import pandas as pd
df = pd.DataFrame(np.array([[]]))
df

""
0


In [53]:
print(np.shape(y_train))
Y_train = []
for i in range(len(y_train)):
    if i % 5 == 0:
        Y_train.append(y_train[i])

print(np.shape(Y_train))
df = pd.DataFrame(Y_train)
Y_train = np.array(df)

(8690, 20)
(1738, 20)


In [54]:
Y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [55]:
df = pd.DataFrame(np.array([[]]))

print(np.shape(y_test))
Y_test = []
for i in range(len(y_test)):
    if i % 5 == 0:
        Y_test.append(y_test[i])

print(np.shape(Y_test))
df = pd.DataFrame(Y_test)
Y_test = np.array(df)

(4970, 20)
(994, 20)


In [56]:
c = custom_vgg_model2.predict(x_test)
d= np.shape(c)
print(d)
c = c.reshape(994,5,20)
print(np.shape(c))
#c = model.predict(predictions)

(4970, 20)
(994, 5, 20)


In [57]:
Y_test

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [58]:
LSTM_Model.fit(a,Y_train,batch_size=32,epochs=10)

Epoch 1/10
1738/1738 [==============================] - 1s 850us/step - loss: 1.1260 - acc: 0.9499
Epoch 2/10
1738/1738 [==============================] - 1s 495us/step - loss: 0.0374 - acc: 0.9925
Epoch 3/10
1738/1738 [==============================] - 1s 495us/step - loss: 0.0095 - acc: 0.9977
Epoch 4/10
1738/1738 [==============================] - 1s 495us/step - loss: 0.0098 - acc: 0.9977
Epoch 5/10
1738/1738 [==============================] - 1s 495us/step - loss: 0.0089 - acc: 0.9983
Epoch 6/10
1738/1738 [==============================] - 1s 497us/step - loss: 0.0054 - acc: 0.9988
Epoch 7/10
1738/1738 [==============================] - 1s 492us/step - loss: 0.0092 - acc: 0.9977
Epoch 8/10
1738/1738 [==============================] - 1s 496us/step - loss: 0.0073 - acc: 0.9983
Epoch 9/10
1738/1738 [==============================] - 1s 491us/step - loss: 0.0054 - acc: 0.9983
Epoch 10/10
1738/1738 [==============================] - 1s 491us/step - loss: 0.0079 - acc: 0.9983


In [59]:
numOfEpochs = 15
loss,acc = LSTM_Model.evaluate(c,Y_test,steps=numOfEpochs)

15/15 [==============================] - 0s 15ms/step


In [60]:
print('Test accuracy:', acc)
#LSTM_Model.save('./UCF_LSTM_MAG.h5')

('Test accuracy:', 0.5824949741363525)


In [61]:
z = LSTM_Model.predict(c)
np.sum(z[0])

1.0

In [62]:
df = pd.DataFrame(np.array([[]]))
df_final = pd.DataFrame(np.array([[]]))

In [63]:
for i in range(len(z)):
    df = pd.DataFrame(z[i])
    df_final = pd.concat([df_final,df],axis  = 1)

In [64]:
#df_final.to_csv("../biswa_n_deba/FINAL_new/MAG_LSTM.csv")

In [77]:
q = LSTM_Model.predict(c)

In [78]:
q = q[0]

In [79]:
r = np.amax(q)
for j in range(20):
    if(q[j]==r):
        q[j]=1
        print(j)
    else:
        q[j]=0


10


In [80]:
q

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [81]:
import requests
def email_alert():
    report = {}
    requests.post("https://maker.ifttt.com/trigger/alert_message/with/key/oXsatCBbGiCbaObQvSfqUSvRWKBApi766mxgs1306vE", data=report)  
if(j==10):
    email_alert()